In [ ]:
import lightcones.linalg as la
from lightcones.linalg import kron

U = 1
t = 0.05

#eps_cut = 0.00001

num_iterations = 100

m = 100
l = 5 + 1

# iteration 0: preparatory

# left subsystem

f = spinfull_fermions(l)

H_hopp = sum([sum([- t * (f.a_dag[s][i] @ f.a[s][i - 1] + f.a_dag[s][i - 1] @ f.a[s][i]) for s in range(2)]) for i in range(l)])
H_S = H_hopp + sum([U * f.n[0][i] @ f.n[1][i] for i in range(l)])

H_S_ = kron(H, f.eye)

a_S = f.a[:][-1]
a_S_dag = f.a_dag[:][-1]

a_S_ = kron(a_S, f.eye)
a_S_dag_ = kron(a_S_dag, f.eye)

# right subsystem

# fermion Hamiltonian is always even
H_E_ = kron(f.eye, H_S)

# a and a_dag are odd operators
a_E_ = kron(f.parity, a_S)
a_E_dag_ = kron(f.parity, a_S_dag)

# joint system Hamiltonian

H_SE = H_S_ + H_E_ - sum([t * (a_E_dag_[s] @ a_S_[s] + a_S_dag_[s] @ a_E_[s]) for s in range(2)])

# now start regular iterations

for i in range(num_iterations):

    # find ground state
    
    from scipy.sparse.linalg import eigsh
    E_GND, Psi_GND = eigsh(H_SE, k=1, which='SM')

    # partial trace

    r = reduce_bipartite(f.states.dimension, f.state_dimension, trace_over = 'R')

    rho = r(Psi_GND)

    # find eigenvalues sorted in ascending order

    pi, T = la.find_eigs_descending(rho)

    # keep only m largest eigenstates
    # columns are eigenstates
    T = T[:, : m] 

    # switch to the frame of revelant eigenstates
    T_dag = T.conj().T
    H_T = T_dag @ H_S @ T

    a_T = [T_dag @ a_S[s] @ T for s in range(2)]
    a_T_dag = [T_dag @ a_S_dag[s] @ T for s in range(2)]

    # new parity operator
    parity_T = T_dag @ f.parity @ T
    eye_T = la.eye(m)
    
    # add new site
    
    g = spinfull_fermions(1)
    
    # a and a_dag are odd operators
    a_S = kron(parity_T, g.a)
    a_S_dag = kron(parity_T, g.a_dag)
    
    a_T_ = kron(a_T, g.eye)
    a_T_dag_ = kron(a_T_dag, g.eye)
    
    # n is even operator
    n_ = kron(eye_T, g.n[:, 0])
        
    H_S = H_T - t * sum([a_S_dag[s] @ a_T_[s] + a_T_dag_[s] @ a_S[s] for s in range(2)]) + U * n_[0] @ n_[1]
    
    eye_T_ = kron(eye_T, g.eye)
    
    # left subsystem
    
    H_S_ = kron(H_S, eye_T_)
    
    a_S_ = kron(a_S, eye_T_)
    a_S_dag_ = kron(a_S_dag, eye_T_)

    # right subsystem

    parity_T_ = kron(parity_T, g.parity)

    # fermion Hamiltonian is always even
    H_E_ = kron(eye_T_, H_S)

    # a and a_dag are odd operators
    a_E_ = kron(parity_T_, a_S)
    a_E_dag_ = kron(parity_T_, a_S_dag)
    
    # joint system Hamiltonian

    H_SE = H_S_ + H_E_ - sum([t * (a_E_dag_[s] @ a_S_[s] + a_S_dag_[s] @ a_E_[s]) for s in range(2)])


    
    
    
    







In [ ]:
f = spinfull_fermions(n_modes)

f.states.state_at()
f.states.index_of()

f.vac()

f.up.a[i]
f.up.a_dag[i]

f.down.a[i]
f.down.a_dag[i]

f.a[s,i] # s = 0 (up) 1 (down)
f.a_dag[s, i]

f.n[s, i]

f.parity

f.zero

f.eye

In [ ]:
import numpy as np

array = np.empty((3, 3), dtype=object)
array[0, 0] = Node(...)